# numerical integration examples

demonstration of various integration methods and accuracy comparison

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from numeric_integrator import integrate, IntegrationResult

print("library loaded successfully")

## example 1: polynomial integration

integrate $\int_0^1 x^2 dx = \frac{1}{3}$

In [ ]:
# define function
f = lambda x: x**2
exact = 1/3

# compare different methods
methods = ["trapezoidal", "simpson", "midpoint", "boole", "romberg"]
results = {}

for method in methods:
    result = integrate(f, 0, 1, method=method, n=100)
    error = abs(result.value - exact)
    results[method] = {"value": result.value, "error": error, "n_evals": result.n_evaluations}
    print(f"{method:15s}: value={result.value:.10f}, error={error:.2e}, evaluations={result.n_evaluations}")

print(f"\nexact value: {exact:.10f}")

## example 2: trigonometric integration

integrate $\int_0^{\pi} \sin(x) dx = 2$

In [ ]:
f = np.sin
exact = 2.0

result_trap = integrate(f, 0, np.pi, method="trapezoidal", n=100)
result_simpson = integrate(f, 0, np.pi, method="simpson", n=100)
result_adaptive = integrate(f, 0, np.pi, method="adaptive_simpson", tol=1e-10)

print(f"trapezoidal: {result_trap.value:.10f}, error={abs(result_trap.value - exact):.2e}")
print(f"simpson:     {result_simpson.value:.10f}, error={abs(result_simpson.value - exact):.2e}")
print(f"adaptive:    {result_adaptive.value:.10f}, error={abs(result_adaptive.value - exact):.2e}")
print(f"\nadaptive used {result_adaptive.n_evaluations} evaluations")

## example 3: convergence analysis

study how error decreases with increasing n

In [ ]:
f = lambda x: np.exp(x)
exact = np.exp(1) - 1

n_values = [10, 20, 50, 100, 200, 500]
errors_trap = []
errors_simpson = []

for n in n_values:
    result_trap = integrate(f, 0, 1, method="trapezoidal", n=n)
    result_simpson = integrate(f, 0, 1, method="simpson", n=n if n%2==0 else n+1)
    
    errors_trap.append(abs(result_trap.value - exact))
    errors_simpson.append(abs(result_simpson.value - exact))

# plot convergence
plt.figure(figsize=(10, 6))
plt.loglog(n_values, errors_trap, 'o-', label='trapezoidal', linewidth=2)
plt.loglog(n_values, errors_simpson, 's-', label='simpson', linewidth=2)
plt.loglog(n_values, [1/n**2 for n in n_values], '--', label='O(h²)', alpha=0.5)
plt.loglog(n_values, [1/n**4 for n in n_values], '--', label='O(h⁴)', alpha=0.5)
plt.xlabel('number of intervals (n)', fontsize=12)
plt.ylabel('absolute error', fontsize=12)
plt.title('convergence of integration methods', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print(f"simpson achieves higher-order convergence (O(h⁴) vs O(h²))")

## example 4: adaptive integration

adaptive methods automatically refine where needed

In [ ]:
# function with sharp feature
def f(x):
    return 1 / (1 + (x - 0.5)**2 * 100)

# uniform vs adaptive
result_uniform = integrate(f, 0, 1, method="simpson", n=100)
result_adaptive = integrate(f, 0, 1, method="adaptive_simpson", tol=1e-6)

print(f"uniform simpson (n=100): value={result_uniform.value:.8f}, evals={result_uniform.n_evaluations}")
print(f"adaptive simpson:        value={result_adaptive.value:.8f}, evals={result_adaptive.n_evaluations}")
print(f"\nadaptive method achieves similar accuracy with {result_adaptive.n_evaluations} vs {result_uniform.n_evaluations} evaluations")

# visualize function
x = np.linspace(0, 1, 500)
y = [f(xi) for xi in x]

plt.figure(figsize=(10, 5))
plt.plot(x, y, linewidth=2)
plt.fill_between(x, 0, y, alpha=0.3)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('function with sharp feature near x=0.5')
plt.grid(True, alpha=0.3)
plt.show()

## example 5: improper-like integrals

integrate $\int_0^1 \frac{1}{\sqrt{x}}dx = 2$

In [ ]:
# add small offset to avoid singularity at x=0
f = lambda x: 1 / np.sqrt(x + 1e-10)
exact = 2.0

result_trap = integrate(f, 0, 1, method="trapezoidal", n=500)
result_adaptive = integrate(f, 0, 1, method="adaptive_simpson", tol=1e-4)

print(f"trapezoidal: {result_trap.value:.6f}, error={abs(result_trap.value - exact):.4f}")
print(f"adaptive:    {result_adaptive.value:.6f}, error={abs(result_adaptive.value - exact):.4f}")
print(f"exact:       {exact:.6f}")